# Capítulo 5: Avaliação de Expressões

## Introdução

O DuckDB avalia expressões SQL de forma **vetorizada** usando uma árvore de expressões pré-compiladas. Diferente de sistemas com JIT (Just-In-Time compilation), o DuckDB usa funções C++ otimizadas que processam vetores inteiros.

### Objetivos:
1. Entender como expressões SQL são representadas em árvores
2. Implementar avaliação vetorizada de expressões
3. Comparar JIT vs interpretação vetorizada
4. Otimizar uso de cache em expressões compostas
5. Entender short-circuit evaluation vetorizado

In [ ]:
!pip install duckdb pandas numpy matplotlib seaborn -q

In [ ]:
import duckdb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from typing import List, Callable, Any, Optional
from dataclasses import dataclass
from enum import Enum

plt.rcParams['figure.figsize'] = (12, 6)

## 5.1 A Árvore de Expressões

Uma expressão SQL como `(colA + colB) * colC` é representada como uma árvore:

```
        [*]              Nível 0 (raiz)
       /   \
     [+]   colC          Nível 1
    /   \
  colA  colB             Nível 2 (folhas)
```

### Avaliação bottom-up:
1. Carrega `colA` e `colB` em vetores
2. Executa `VectorAdd(colA, colB)` → `tmp1` (fica no cache!)
3. Executa `VectorMultiply(tmp1, colC)` → resultado

### Por que não JIT?
```
JIT (Spark SQL, PrestoSQL):
  + Código compilado específico para a query
  - Tempo de compilação (latência)
  - Complexidade de implementação

Interpretação Vetorizada (DuckDB):
  + Sem tempo de compilação
  + Funções pré-otimizadas em C++
  + Dados ficam quentes no cache entre operações
```

In [ ]:
# Implementação de uma árvore de expressões completa

class ExprType(Enum):
    # Literais
    COLUMN = "COLUMN"
    CONSTANT = "CONSTANT"
    
    # Operações aritméticas
    ADD = "ADD"
    SUBTRACT = "SUBTRACT"
    MULTIPLY = "MULTIPLY"
    DIVIDE = "DIVIDE"
    MODULO = "MODULO"
    
    # Comparações
    EQUAL = "EQUAL"
    NOT_EQUAL = "NOT_EQUAL"
    GREATER_THAN = "GREATER_THAN"
    LESS_THAN = "LESS_THAN"
    GREATER_EQUAL = "GREATER_EQUAL"
    LESS_EQUAL = "LESS_EQUAL"
    
    # Lógicas
    AND = "AND"
    OR = "OR"
    NOT = "NOT"
    
    # Funções
    ABS = "ABS"
    SQRT = "SQRT"
    CASE = "CASE"

@dataclass
class Expression:
    expr_type: ExprType
    value: Any = None       # Para COLUMN (nome) ou CONSTANT (valor)
    left: 'Expression' = None
    right: 'Expression' = None
    condition: 'Expression' = None  # Para CASE WHEN
    
    def __repr__(self):
        if self.expr_type == ExprType.COLUMN:
            return f"Col({self.value})"
        elif self.expr_type == ExprType.CONSTANT:
            return f"{self.value}"
        elif self.left and self.right:
            return f"({self.left} {self.expr_type.value} {self.right})"
        return f"{self.expr_type.value}"

# Funções helper para criar expressões
def col(name: str) -> Expression:
    return Expression(ExprType.COLUMN, value=name)

def const(value) -> Expression:
    return Expression(ExprType.CONSTANT, value=value)

def add(left, right) -> Expression:
    return Expression(ExprType.ADD, left=left, right=right)

def sub(left, right) -> Expression:
    return Expression(ExprType.SUBTRACT, left=left, right=right)

def mul(left, right) -> Expression:
    return Expression(ExprType.MULTIPLY, left=left, right=right)

def div(left, right) -> Expression:
    return Expression(ExprType.DIVIDE, left=left, right=right)

def gt(left, right) -> Expression:
    return Expression(ExprType.GREATER_THAN, left=left, right=right)

def lt(left, right) -> Expression:
    return Expression(ExprType.LESS_THAN, left=left, right=right)

def eq(left, right) -> Expression:
    return Expression(ExprType.EQUAL, left=left, right=right)

def and_(left, right) -> Expression:
    return Expression(ExprType.AND, left=left, right=right)

def or_(left, right) -> Expression:
    return Expression(ExprType.OR, left=left, right=right)

# Exemplo: (colA + colB) * colC
expr = mul(add(col('A'), col('B')), col('C'))
print(f"Expressão: {expr}")

# Exemplo mais complexo: (A > 50) AND ((B + C) < 100)
expr_complex = and_(gt(col('A'), const(50)), lt(add(col('B'), col('C')), const(100)))
print(f"Expressão complexa: {expr_complex}")

## 5.2 Avaliador Vetorizado

In [ ]:
class VectorizedExpressionEvaluator:
    """Avalia expressões de forma vetorizada (como o DuckDB)"""
    
    def __init__(self, data: dict):
        self.data = data  # Dict de nome_coluna -> np.array
        self.operation_count = 0
    
    def evaluate(self, expr: Expression) -> np.ndarray:
        """Avalia expressão de forma bottom-up vetorizada"""
        self.operation_count += 1
        
        # Literais
        if expr.expr_type == ExprType.COLUMN:
            return self.data[expr.value]
        
        elif expr.expr_type == ExprType.CONSTANT:
            size = len(next(iter(self.data.values())))
            return np.full(size, expr.value)
        
        # Operações aritméticas - todas vetorizadas!
        elif expr.expr_type == ExprType.ADD:
            return self.evaluate(expr.left) + self.evaluate(expr.right)
        
        elif expr.expr_type == ExprType.SUBTRACT:
            return self.evaluate(expr.left) - self.evaluate(expr.right)
        
        elif expr.expr_type == ExprType.MULTIPLY:
            return self.evaluate(expr.left) * self.evaluate(expr.right)
        
        elif expr.expr_type == ExprType.DIVIDE:
            right = self.evaluate(expr.right)
            # Evitar divisão por zero
            return np.where(right != 0, self.evaluate(expr.left) / right, 0)
        
        # Comparações
        elif expr.expr_type == ExprType.GREATER_THAN:
            return self.evaluate(expr.left) > self.evaluate(expr.right)
        
        elif expr.expr_type == ExprType.LESS_THAN:
            return self.evaluate(expr.left) < self.evaluate(expr.right)
        
        elif expr.expr_type == ExprType.EQUAL:
            return self.evaluate(expr.left) == self.evaluate(expr.right)
        
        elif expr.expr_type == ExprType.GREATER_EQUAL:
            return self.evaluate(expr.left) >= self.evaluate(expr.right)
        
        elif expr.expr_type == ExprType.LESS_EQUAL:
            return self.evaluate(expr.left) <= self.evaluate(expr.right)
        
        # Operações lógicas
        elif expr.expr_type == ExprType.AND:
            return self.evaluate(expr.left) & self.evaluate(expr.right)
        
        elif expr.expr_type == ExprType.OR:
            return self.evaluate(expr.left) | self.evaluate(expr.right)
        
        elif expr.expr_type == ExprType.NOT:
            return ~self.evaluate(expr.left)
        
        raise ValueError(f"Unknown expression type: {expr.expr_type}")

# Teste
size = 10000
data = {
    'A': np.random.randn(size) * 100,
    'B': np.random.randn(size) * 100,
    'C': np.random.randn(size) * 100
}

evaluator = VectorizedExpressionEvaluator(data)

# (A + B) * C
result = evaluator.evaluate(expr)
print(f"Expressão: (A + B) * C")
print(f"Resultado (primeiros 5): {result[:5]}")
print(f"Verificação manual: {(data['A'][:5] + data['B'][:5]) * data['C'][:5]}")
print(f"Operações realizadas: {evaluator.operation_count}")

In [ ]:
# Teste com expressão complexa
evaluator2 = VectorizedExpressionEvaluator(data)

# (A > 50) AND ((B + C) < 100)
result_complex = evaluator2.evaluate(expr_complex)
print(f"\nExpressão: (A > 50) AND ((B + C) < 100)")
print(f"Resultados True: {np.sum(result_complex)} de {len(result_complex)}")
print(f"Operações realizadas: {evaluator2.operation_count}")

## 5.3 Cache de Resultados Intermediários

**Por que funciona**: Resultados intermediários ficam no cache L1/L2, prontos para a próxima operação!

```
Expressão: (A + B) * C

Passo 1: tmp = A + B
  - Carrega A e B da RAM → Cache L1
  - Resultado tmp vai para Cache L1

Passo 2: result = tmp * C
  - tmp JÁ ESTÁ no Cache L1! (hit)
  - Carrega C da RAM → Cache L1
  - Resultado final

Benefício: Evita um round-trip para RAM!
```

In [ ]:
# Demonstração: Localidade de cache em expressões compostas

def evaluate_expression_row_by_row(A, B, C):
    """Avalia (A + B) * C linha por linha - BAD for cache"""
    result = np.zeros_like(A)
    for i in range(len(A)):
        result[i] = (A[i] + B[i]) * C[i]
    return result

def evaluate_expression_vectorized(A, B, C):
    """Avalia (A + B) * C vetorizado - GOOD for cache"""
    tmp = A + B  # Resultado fica no cache!
    return tmp * C  # Usa dados do cache

def evaluate_expression_fused(A, B, C):
    """Avalia com fusão de operações (como NumPy otimiza)"""
    return (A + B) * C  # NumPy pode otimizar internamente

# Benchmark
size = 1_000_000
A = np.random.randn(size)
B = np.random.randn(size)
C = np.random.randn(size)

# Row by row
start = time.perf_counter()
r1 = evaluate_expression_row_by_row(A, B, C)
row_time = time.perf_counter() - start

# Vectorized
start = time.perf_counter()
for _ in range(10):  # Repetir para medir melhor
    r2 = evaluate_expression_vectorized(A, B, C)
vec_time = (time.perf_counter() - start) / 10

# Fused
start = time.perf_counter()
for _ in range(10):
    r3 = evaluate_expression_fused(A, B, C)
fused_time = (time.perf_counter() - start) / 10

print(f"Expressão: (A + B) * C em {size:,} elementos")
print(f"Row-by-row:  {row_time*1000:8.2f} ms")
print(f"Vetorizado:  {vec_time*1000:8.4f} ms")
print(f"Fused:       {fused_time*1000:8.4f} ms")
print(f"\nSpeedup row→vec: {row_time/vec_time:.0f}x")

## 5.4 Avaliação por Chunks

Para dados muito grandes, processa-se por chunks para manter tudo no cache L1.

In [ ]:
def evaluate_chunked(A, B, C, chunk_size=2048):
    """
    Avalia expressão por chunks para melhor uso de cache.
    Chunks de 2048 elementos × 8 bytes = 16KB (cabe no L1 de 32KB)
    """
    n = len(A)
    result = np.zeros(n)
    
    for start in range(0, n, chunk_size):
        end = min(start + chunk_size, n)
        # Processa chunk inteiro - dados ficam no L1
        chunk_a = A[start:end]
        chunk_b = B[start:end]
        chunk_c = C[start:end]
        
        tmp = chunk_a + chunk_b  # No cache L1
        result[start:end] = tmp * chunk_c  # Ainda no cache!
    
    return result

# Benchmark com diferentes tamanhos de chunk
sizes = [256, 512, 1024, 2048, 4096, 8192, 16384, 32768]
times = []

# Usar array maior para ver diferença
size = 10_000_000
A = np.random.randn(size)
B = np.random.randn(size)
C = np.random.randn(size)

for cs in sizes:
    t = []
    for _ in range(5):
        start = time.perf_counter()
        _ = evaluate_chunked(A, B, C, cs)
        t.append(time.perf_counter() - start)
    times.append(np.mean(t) * 1000)
    mem_kb = cs * 8 * 3 / 1024  # 3 arrays × 8 bytes
    print(f"Chunk size {cs:>6}: {times[-1]:6.2f} ms  (mem: {mem_kb:.0f} KB)")

In [ ]:
# Visualização
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Tempo vs chunk size
ax1.plot(sizes, times, marker='o', linewidth=2)
ax1.axvline(x=2048, color='red', linestyle='--', label='DuckDB default (2048)')
ax1.set_xlabel('Tamanho do Chunk')
ax1.set_ylabel('Tempo (ms)')
ax1.set_title('Impacto do Tamanho do Chunk na Performance')
ax1.set_xscale('log', base=2)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Memória vs cache levels
mem_usage = [cs * 8 * 3 / 1024 for cs in sizes]  # KB
ax2.bar(range(len(sizes)), mem_usage, color='steelblue')
ax2.axhline(y=32, color='red', linestyle='--', linewidth=2, label='L1 Cache (32 KB)')
ax2.axhline(y=256, color='orange', linestyle='--', linewidth=2, label='L2 Cache (256 KB)')
ax2.set_xticks(range(len(sizes)))
ax2.set_xticklabels(sizes, rotation=45)
ax2.set_xlabel('Tamanho do Chunk')
ax2.set_ylabel('Memória por Chunk (KB)')
ax2.set_title('Uso de Memória vs Níveis de Cache')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5.5 Short-Circuit Evaluation Vetorizado

Em expressões com AND/OR, podemos evitar computar partes desnecessárias:

```
WHERE (A > 100) AND (expensive_function(B))

Tradicional: Avalia ambos para todas as linhas
Otimizado:   Se A <= 100, nem avalia expensive_function!
```

In [ ]:
def expensive_function(arr):
    """Simula função cara (ex: regex, cálculo complexo)"""
    time.sleep(0.001)  # Simula processamento
    return np.sqrt(np.abs(arr)) * np.sin(arr)

def evaluate_no_shortcircuit(A, B):
    """Avalia (A > 100) AND expensive(B) sem short-circuit"""
    cond1 = A > 100
    cond2 = expensive_function(B) > 0  # Sempre avalia tudo!
    return cond1 & cond2

def evaluate_with_shortcircuit(A, B):
    """Avalia (A > 100) AND expensive(B) com short-circuit"""
    cond1 = A > 100
    
    # Só avalia expensive para linhas onde cond1 é True
    result = np.zeros(len(A), dtype=bool)
    mask = cond1
    
    if np.any(mask):
        # Avalia só para o subset
        cond2 = expensive_function(B[mask]) > 0
        result[mask] = cond2
    
    return result

# Teste com dados onde poucos passam no primeiro filtro
n = 10000
A = np.random.randn(n) * 10  # Poucos > 100!
B = np.random.randn(n)

print(f"Linhas onde A > 100: {np.sum(A > 100)} de {n}")

start = time.perf_counter()
r1 = evaluate_no_shortcircuit(A, B)
no_sc_time = time.perf_counter() - start

start = time.perf_counter()
r2 = evaluate_with_shortcircuit(A, B)
sc_time = time.perf_counter() - start

print(f"\nSem short-circuit: {no_sc_time*1000:.2f} ms")
print(f"Com short-circuit: {sc_time*1000:.2f} ms")
print(f"Speedup: {no_sc_time/sc_time:.1f}x")

## 5.6 DuckDB: Expressões Complexas em Ação

In [ ]:
con = duckdb.connect(':memory:')

# Criar tabela de teste
con.execute("""
    CREATE TABLE expressions AS
    SELECT 
        random() * 100 AS a,
        random() * 100 AS b,
        random() * 100 AS c,
        random() * 100 AS d,
        (random() * 1000)::INTEGER AS category
    FROM generate_series(1, 10000000)
""")

print("Tabela criada com 10M linhas")

In [ ]:
# Benchmark de expressões de diferentes complexidades
expressions = {
    'Simples (a + b)': 'SELECT SUM(a + b) FROM expressions',
    'Média ((a + b) * c)': 'SELECT SUM((a + b) * c) FROM expressions',
    'Complexa (a+b)*c - d/(a+1)': 'SELECT SUM((a + b) * c - d / (a + 1)) FROM expressions',
    'Com CASE simples': 'SELECT SUM(CASE WHEN a > 50 THEN b ELSE c END) FROM expressions',
    'Com CASE complexo': 'SELECT SUM(CASE WHEN a > 50 THEN b * c WHEN a > 25 THEN b + c ELSE d END) FROM expressions',
    'Múltiplas agregações': 'SELECT SUM(a*b), SUM(b*c), SUM(c*d), AVG(a+b+c+d) FROM expressions',
    'Com filtro AND': 'SELECT SUM(a) FROM expressions WHERE a > 50 AND b > 50',
    'Com filtro OR': 'SELECT SUM(a) FROM expressions WHERE a > 90 OR b > 90',
    'Expressão aninhada': 'SELECT SUM(SQRT(ABS(a - b)) * LOG(c + 1)) FROM expressions',
}

print("Benchmark de Expressões:\n")
results = []
for name, query in expressions.items():
    times = []
    for _ in range(5):
        start = time.perf_counter()
        con.execute(query).fetchall()
        times.append(time.perf_counter() - start)
    avg_time = np.mean(times) * 1000
    results.append({'expression': name, 'time_ms': avg_time})
    print(f"{name:35} {avg_time:8.2f} ms")

In [ ]:
# Visualização
df = pd.DataFrame(results)

plt.figure(figsize=(12, 6))
bars = plt.barh(df['expression'], df['time_ms'], color='steelblue')
plt.xlabel('Tempo (ms)')
plt.title('Performance de Diferentes Expressões no DuckDB (10M linhas)')
plt.grid(True, alpha=0.3, axis='x')

# Adicionar valores nas barras
for bar, val in zip(bars, df['time_ms']):
    plt.text(val + 1, bar.get_y() + bar.get_height()/2, f'{val:.1f}ms', va='center')

plt.tight_layout()
plt.show()

In [ ]:
# Ver plano de execução
print("=== EXPLAIN ===")
plan = con.execute("EXPLAIN SELECT (a + b) * c - d / (a + 1) FROM expressions").df()
print(plan.to_string())

In [ ]:
# Ver plano com estatísticas
print("\n=== EXPLAIN ANALYZE ===")
plan = con.execute("""
    EXPLAIN ANALYZE
    SELECT SUM(CASE WHEN a > 50 THEN b * c ELSE d END)
    FROM expressions
    WHERE a > 25
""").df()
print(plan.to_string())

## 5.7 Comparação: DuckDB vs Pandas

In [ ]:
# Criar DataFrame Pandas equivalente
df_pandas = con.execute("SELECT a, b, c, d FROM expressions").df()
print(f"DataFrame: {len(df_pandas):,} linhas")

# Benchmark: Expressão complexa
expression = "(a + b) * c - d / (a + 1)"

# DuckDB
times_duck = []
for _ in range(5):
    start = time.perf_counter()
    con.execute(f"SELECT SUM({expression}) FROM expressions").fetchall()
    times_duck.append(time.perf_counter() - start)

# Pandas
times_pandas = []
for _ in range(5):
    start = time.perf_counter()
    result = ((df_pandas['a'] + df_pandas['b']) * df_pandas['c'] - df_pandas['d'] / (df_pandas['a'] + 1)).sum()
    times_pandas.append(time.perf_counter() - start)

print(f"\nExpressão: {expression}")
print(f"DuckDB: {np.mean(times_duck)*1000:.2f} ms")
print(f"Pandas: {np.mean(times_pandas)*1000:.2f} ms")
print(f"Speedup: {np.mean(times_pandas)/np.mean(times_duck):.1f}x")

## 5.8 Resumo

| Aspecto | Abordagem Tradicional | DuckDB Vetorizado |
|---------|----------------------|-------------------|
| **Unidade de processamento** | 1 linha por vez | 2048 linhas (chunk) |
| **Cache** | Cold (misses frequentes) | Hot (resultados no L1) |
| **Expressões** | Avaliam completo por linha | Operador por operador |
| **SIMD** | Não usa | Usa extensivamente |
| **Short-circuit** | Por linha | Por chunk com máscaras |
| **Compilação** | JIT (latência) | Pré-compilado (sem latência) |

### Princípios Chave:

1. **Árvore de expressões**: Representação que permite avaliação bottom-up
2. **Chunks de 2048**: Dados cabem no cache L1
3. **Resultados intermediários**: Ficam quentes no cache
4. **Short-circuit vetorizado**: Máscaras evitam computação desnecessária
5. **Funções pré-compiladas**: VectorAdd, VectorMultiply, etc.

In [ ]:
con.close()